https://sarinsuriyakoon.medium.com/use-ollama-to-run-seallm-model-southeast-asia-ff13045be6f7

https://github.com/ollama/ollama/blob/main/examples/

In [1]:
import os
os.getcwd()

'c:\\Workspace\\Coding\\RAG_QA_enth\\LLM'

In [2]:
from langchain import PromptTemplate
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
# LLM_MODEL="nxphi47/seallm-7b-v2:q4_0"
LLM_MODEL = "SeaLLM"

## simple generatation

In [3]:
llm = Ollama(model=LLM_MODEL)

In [4]:
# prompt= input("What is your question?")
prompt= "กาลครั้งหนึ่งนานมาแล้ว"
res = llm.predict(prompt)
res

c:\Workspace\Coding\RAG_QA_enth\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' แต่ในที่สุด ราฟาเอลก็ได้นำมาฝากกับคนรักของเขา ในขณะที่พวกเขากำลังเตรียมตัวไปปาร์ตี้ การกระทำของแฟนหนุ่มทำให้ผู้คนรอบข้างต้องหวาดกลัว และเขาจำเป็นต้องกู้ยาพิษเพื่อรักษาชีวิตของเขา\nดังนั้น เมื่อเวลาผ่านไป ราฟาเอลต้องเผชิญกับความเจ็บปวดที่รุนแรงขณะที่เขาพยายามที่จะกักกันอวัยวะที่เสียหายและรักษาชีวิตของเขาไว้ โดยใช้เวลาหลายปีในการฟื้นตัว หลังจากได้รับการบาดเจ็บจากยาพิษ และในที่สุดก็กลับไปเป็นปกติอีกครั้ง\nคำอธิบาย: การสะบั่นไส้เกิดขึ้นเมื่อคนคนนั้นถูกทอดทิ้งโดยแฟนหนุ่ม และต้องเผชิญกับความเจ็บปวดที่รุนแรงขณะพยายามฟื้นตัว'

## Web Summary


In [5]:
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

In [6]:
loader = WebBaseLoader("https://th.wikipedia.org/wiki/%E0%B8%A0%E0%B8%B2%E0%B8%A9%E0%B8%B2%E0%B8%81%E0%B8%A7%E0%B8%B2%E0%B8%87%E0%B8%95%E0%B8%B8%E0%B9%89%E0%B8%87")
docs = loader.load()

In [7]:
llm = Ollama(model=LLM_MODEL)
chain = load_summarize_chain(llm, chain_type="stuff")


In [8]:
result = chain.run(docs)
print(result)

c:\Workspace\Coding\RAG_QA_enth\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ภาษากวางตุ้งคือภาษาของชาวกวางตุ้งในประเทศไทย มีการพัฒนาระบบการเขียนที่แตกต่างจากภาษาจีนกลาง โดยใช้อักษรจีนแบบเดียวกับภาษาจีนกลาง แต่อ่านออกเสียงแบบภาษากวางตุ้ง

คำอ่าน: 粤语 (Yuèyǔ) หรือ Cantonese


In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [10]:
from sentence_transformers import SentenceTransformer, util

In [11]:
# load the document and split it into chunks
loader = TextLoader("./doc/sayaka.txt", encoding='utf8')
documents = loader.load()

In [12]:
# split it into chunks
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# implement RAG

In [13]:
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import sys

In [14]:
embedding = FastEmbedEmbeddings()

c:\Workspace\Coding\RAG_QA_enth\.venv\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
# vector_store = Chroma(persist_directory="./chroma_db", 
#             embedding_function=embedding)
vectordb = Chroma.from_documents(documents=docs, 
                                        embedding=embedding,
                                        persist_directory="./chroma_db")

In [16]:
vectordb.persist()

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [18]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [19]:
prompt = "who is sayaka?"
response = qa(prompt)
response

c:\Workspace\Coding\RAG_QA_enth\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [21]:
prompt = "ใครคือ sayaka?"
response = qa(prompt)
response 

{'query': 'ใครคือ sayaka?',
 'result': ' Sayaka Yamamoto เป็นนักร้องและนักเขียนเพลงชาวญี่ปุ่นที่ทำงานให้กับ Universal Sigma เธอเคยเป็นสมาชิกของวง NMB48 และ AKB48 Team K และมีผลการขายสูงในชาร์ต Oricon'}